In [ ]:
!pip install transformers cache_decorator pytorch_lightning

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from drive.MyDrive.historical_events.irproject.historical_events import (
    load_data, preprocess_data 
)
from drive.MyDrive.historical_events.irproject.models import (
    FreezeCallback, MultiTaskLearningModel, WikiDataModule,
    BiLSTM_clf, Bert_clf
)

In [1]:
from pytorch_lightning import Trainer, seed_everything
from pytorch_lightning.callbacks.model_checkpoint import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger

seed_everything(42, workers=True)

Global seed set to 42


42

In [2]:
from irproject.models import (
    FreezeCallback, MultiTaskLearningModel, WikiDataModule,
    BiLSTM_clf, Bert_clf
)

This cell can be skipped if data has been already saved in the train, validation and test splits.

In [ ]:
import torch
import json
import spacy
from sklearn.model_selection import train_test_split
from transformers import BertTokenizerFast

spacy.prefer_gpu()
nlp = spacy.load("en_core_web_sm")

with open(
    f"datasets/historical_events/wiki_dataset.json",
    encoding="utf-8"
) as f_in:
    dataset = json.load(f_in)

texts, tags, labels = load_data(dataset, nlp)
train_texts, test_texts, train_tags, test_tags, train_labels, test_labels = train_test_split(
    texts, tags, labels, test_size=0.2, random_state=42, stratify=labels
)
train_texts, valid_texts, train_tags, valid_tags, train_labels, valid_labels = train_test_split(
    train_texts, train_tags, train_labels, test_size=0.2, random_state=42, stratify=train_labels
)

tokenizer = BertTokenizerFast.from_pretrained("bert-base-cased")

encodings, tokens_labels, labels, tag2idx, idx2tag = preprocess_data(
    train_texts.tolist(), train_tags.tolist(), 
    train_labels.tolist(), tokenizer, padding="max_length"
)
torch.save(encodings.input_ids, "datasets/historical_events/train/input_ids.pkl")
torch.save(encodings.attention_mask, "datasets/historical_events/train/attention_mask.pkl")
torch.save(tokens_labels, "datasets/historical_events/train/tokens_labels.pkl")
torch.save(labels, "datasets/historical_events/train/labels.pkl")
torch.save(tag2idx, "datasets/historical_events/train/tag2idx.pkl")
torch.save(idx2tag, "datasets/historical_events/train/idx2tag.pkl")

encodings, tokens_labels, labels, tag2idx, idx2tag = preprocess_data(
    valid_texts.tolist(), valid_tags.tolist(), 
    valid_labels.tolist(), tokenizer, padding="max_length"
)
torch.save(encodings.input_ids, "datasets/historical_events/valid/input_ids.pkl")
torch.save(encodings.attention_mask, "datasets/historical_events/valid/attention_mask.pkl")
torch.save(tokens_labels, "datasets/historical_events/valid/tokens_labels.pkl")
torch.save(labels, "datasets/historical_events/valid/labels.pkl")
torch.save(tag2idx, "datasets/historical_events/valid/tag2idx.pkl")
torch.save(idx2tag, "datasets/historical_events/valid/idx2tag.pkl")

encodings, tokens_labels, labels, tag2idx, idx2tag = preprocess_data(
    test_texts.tolist(), test_tags.tolist(), 
    test_labels.tolist(), tokenizer, padding="max_length"
)
torch.save(encodings.input_ids, "datasets/historical_events/test/input_ids.pkl")
torch.save(encodings.attention_mask, "datasets/historical_events/test/attention_mask.pkl")
torch.save(tokens_labels, "datasets/historical_events/test/tokens_labels.pkl")
torch.save(labels, "datasets/historical_events/test/labels.pkl")
torch.save(tag2idx, "datasets/historical_events/test/tag2idx.pkl")
torch.save(idx2tag, "datasets/historical_events/test/idx2tag.pkl")

In [8]:
checkpoint_callback = ModelCheckpoint(
    #dirpath="/content/drive/MyDrive/historical_events/checkpoints",
    dirpath="checkpoints/historical_events/mtl",
    every_n_epochs=1
)

freeze_callback = FreezeCallback()

D:\Users\gabri\anaconda3\envs\tf_p3.9\lib\site-packages\pytorch_lightning\callbacks\model_checkpoint.py:446: UserWarning: Checkpoint directory checkpoints/historical_events/mtl exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")


In [ ]:
dm = WikiDataModule(
    batch_size=4, 
    # data_dir="/content/drive/MyDrive/historical_events/datasets/historical_events"
)
model = MultiTaskLearningModel()
logger = TensorBoardLogger(
    "tb_logs", name="mtl"
)
trainer = Trainer(
    gpus=1,
    deterministic=True,
    logger=logger,
    callbacks=[checkpoint_callback, freeze_callback]
)
trainer.fit(model, dm)

In [9]:
dm = WikiDataModule(
    batch_size=4, 
    # data_dir="/content/drive/MyDrive/historical_events/datasets/historical_events"
)
model = MultiTaskLearningModel().load_from_checkpoint(
    "checkpoints/historical_events/mtl/epoch=13-step=45415.ckpt"
)
logger = TensorBoardLogger(
    "tb_logs", name="mtl"
)
trainer = Trainer(
    gpus=1,
    deterministic=True,
    logger=logger,
    # callbacks=[checkpoint_callback, freeze_callback],
    resume_from_checkpoint="checkpoints/historical_events/mtl/epoch=13-step=45415.ckpt"
)
trainer.test(
    model, 
    dm,
    ckpt_path="checkpoints/historical_events/mtl/epoch=13-step=45415.ckpt"
)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.L

Testing: 0it [00:00, ?it/s]

D:\Users\gabri\anaconda3\envs\tf_p3.9\lib\site-packages\pytorch_lightning\trainer\data_loading.py:105: UserWarning: The dataloader, test dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Testing: 100%|██████████| 2027/2027 [03:15<00:00, 10.34it/s]--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_seqc_acc': 0.8536013960838318,
 'test_seqc_acc_epoch': 0.8536013960838318,
 'test_seqc_f1': 0.7163200974464417,
 'test_seqc_f1_epoch': 0.7163200974464417,
 'test_tokc_acc': 0.8814392685890198,
 'test_tokc_acc_epoch': 0.8814392685890198,
 'test_tokc_f1': 0.03747977688908577,
 'test_tokc_f1_epoch': 0.03747977688908577}
--------------------------------------------------------------------------------
Testing: 100%|██████████| 2027/2027 [03:15<00:00, 10.36it/s]


[{'test_seqc_acc': 0.8536013960838318,
  'test_seqc_acc_epoch': 0.8536013960838318,
  'test_tokc_acc': 0.8814392685890198,
  'test_tokc_acc_epoch': 0.8814392685890198,
  'test_seqc_f1': 0.7163200974464417,
  'test_seqc_f1_epoch': 0.7163200974464417,
  'test_tokc_f1': 0.03747977688908577,
  'test_tokc_f1_epoch': 0.03747977688908577}]

In [3]:
checkpoint_callback = ModelCheckpoint(
    dirpath="checkpoints/historical_events/bilstm_clf",
    every_n_epochs=1
)

D:\Users\gabri\anaconda3\envs\tf_p3.9\lib\site-packages\pytorch_lightning\callbacks\model_checkpoint.py:446: UserWarning: Checkpoint directory checkpoints/historical_events/bilstm_clf exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")


In [ ]:
dm = WikiDataModule(
    batch_size=4, 
    # data_dir="/content/drive/MyDrive/historical_events/datasets/historical_events"
)
model = BiLSTM_clf(
    dropout_rate=0.5,
    freeze_bert=True
)
logger = TensorBoardLogger(
    "tb_logs", name="bilstm_clf"
)
trainer = Trainer(
    gpus=1,
    deterministic=True,
    precision=16,
    gradient_clip_val=1,
    logger=logger,
    max_epochs=1,
    callbacks=[checkpoint_callback],
    # resume_from_checkpoint="checkpoints/historical_events/bilstm_clf/epoch=7-step=51895.ckpt"
)
trainer.fit(model, dm)

In [4]:
dm = WikiDataModule(
    batch_size=4, 
    # data_dir="/content/drive/MyDrive/historical_events/datasets/historical_events"
)
model = BiLSTM_clf(
    dropout_rate=0.5,
    freeze_bert=True
).load_from_checkpoint(
    "checkpoints/historical_events/bilstm_clf/epoch=7-step=51895.ckpt",
    dropout_rate=0.5,
    freeze_bert=True
)
logger = TensorBoardLogger(
    "tb_logs", name="bilstm_clf"
)
trainer = Trainer(
    gpus=1,
    deterministic=True,
    precision=16,
    gradient_clip_val=1,
    logger=logger,
    callbacks=[checkpoint_callback],
    resume_from_checkpoint="checkpoints/historical_events/bilstm_clf/epoch=7-step=51895.ckpt"
)
trainer.test(
    model, 
    dm,
    ckpt_path="checkpoints/historical_events/bilstm_clf/epoch=7-step=51895.ckpt"
)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.L

Testing: 0it [00:00, ?it/s]

D:\Users\gabri\anaconda3\envs\tf_p3.9\lib\site-packages\pytorch_lightning\trainer\data_loading.py:105: UserWarning: The dataloader, test dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Testing: 100%|██████████| 2027/2027 [03:46<00:00,  8.59it/s]--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': 0.7935372591018677,
 'test_acc_epoch': 0.7935372591018677,
 'test_f1': 0.6341437697410583,
 'test_f1_epoch': 0.6341437697410583}
--------------------------------------------------------------------------------
Testing: 100%|██████████| 2027/2027 [03:46<00:00,  8.93it/s]


[{'test_acc': 0.7935372591018677,
  'test_acc_epoch': 0.7935372591018677,
  'test_f1': 0.6341437697410583,
  'test_f1_epoch': 0.6341437697410583}]

In [6]:
checkpoint_callback = ModelCheckpoint(
    dirpath="checkpoints/historical_events/bert_clf",
    every_n_epochs=1,
    save_top_k=2,
    monitor="valid_f1",
    mode="max"
)

In [ ]:
dm = WikiDataModule(
    batch_size=4, 
    # data_dir="/content/drive/MyDrive/historical_events/datasets/historical_events"
)
model = Bert_clf(
    dropout_rate=0.5,
    freeze_bert=False
)
logger = TensorBoardLogger(
    "tb_logs", name="bert_clf"
)
trainer = Trainer(
    gpus=1,
    deterministic=True,
    precision=16,
    gradient_clip_val=1, 
    logger=logger,
    max_epochs=1,
    callbacks=[checkpoint_callback]
)
trainer.fit(model, dm)

In [7]:
dm = WikiDataModule(
    batch_size=4, 
    # data_dir="/content/drive/MyDrive/historical_events/datasets/historical_events"
)
model = Bert_clf(
    dropout_rate=0.5,
    freeze_bert=False
).load_from_checkpoint(
    "checkpoints/historical_events/bert_clf/epoch=0-step=6486.ckpt",
    dropout_rate=0.5,
    freeze_bert=False
)
logger = TensorBoardLogger(
    "tb_logs", name="bert_clf"
)
trainer = Trainer(
    gpus=1,
    deterministic=True,
    precision=16,
    gradient_clip_val=1, 
    logger=logger,
    max_epochs=1,
    callbacks=[checkpoint_callback]
)
trainer.test(
    model, 
    dm,
    ckpt_path="checkpoints/historical_events/bert_clf/epoch=0-step=6486.ckpt"
)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.L

Testing: 100%|█████████▉| 2025/2027 [01:37<00:00, 20.52it/s]--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': 0.7988406419754028,
 'test_acc_epoch': 0.7988406419754028,
 'test_f1': 0.6298896670341492,
 'test_f1_epoch': 0.6298896670341492}
--------------------------------------------------------------------------------
Testing: 100%|██████████| 2027/2027 [01:37<00:00, 20.72it/s]


[{'test_acc': 0.7988406419754028,
  'test_acc_epoch': 0.7988406419754028,
  'test_f1': 0.6298896670341492,
  'test_f1_epoch': 0.6298896670341492}]